In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv(r"C:\Users\humza\OneDrive - Worcester Polytechnic Institute (wpi.edu)\mqp\Data\Refinitiv ESG - Final Data for Analysis(Final_Table).csv")

# Delete rows containing the value 'Unknown' 
data = data[~data.eq('Unknown').any(axis=1)]

# Columns to clean
columns_to_clean = [
    'CSR Strategy Score', 'Community Score', 'ESG Combined Score', 'ESG Controversies Score', 'ESG Score',
    'Emissions Score', 'Environment Pillar Score', 'Environmental Innovation Score', 'Governance Pillar Score',
    'Human Rights Score', 'Management Score', 'Product Responsibility Score', 'Resource Use Score',
    'Shareholders Score', 'Social Pillar Score', 'TRDIR Controversies Score', 'TRDIR Diversity Score',
    'TRDIR Inclusion Score', 'TRDIR People Development Score', 'TRDIR Score', 'Workforce Score',
    'Total Returns', 'Standard Deviation'
]

# Convert all columns in columns_to_clean to float type
for col in columns_to_clean:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Averaging 'Total Returns' and other scores for each unique value in the 'Symbol' column
columns_to_average = [
    'CSR Strategy Score', 'Community Score', 'ESG Combined Score', 'ESG Controversies Score', 'ESG Score',
    'Emissions Score', 'Environment Pillar Score', 'Environmental Innovation Score', 'Governance Pillar Score',
    'Human Rights Score', 'Management Score', 'Product Responsibility Score', 'Resource Use Score',
    'Shareholders Score', 'Social Pillar Score', 'TRDIR Controversies Score', 'TRDIR Diversity Score',
    'TRDIR Inclusion Score', 'TRDIR People Development Score', 'TRDIR Score', 'Workforce Score',
    'Total Returns', 'Standard Deviation'
]

# Group by 'Symbol' and calculate the mean for the specified columns
averaged_data = data.groupby('Symbol')[columns_to_average].mean().reset_index()

# Rename 'Total Returns' to 'Predicted Total Returns' and multiply values by 10 and add %
averaged_data['Predicted Total Returns'] = averaged_data['Total Returns'] * 10
averaged_data['Predicted Total Returns'] = averaged_data['Predicted Total Returns'].apply(lambda x: f"{x:.2f}%")

# User responses to the questionnaire 
user_preferences = {
    'How important is environmental protection to you': 5,  # Question 1
    'How important are human rights protection to you': 4,   # Question 2
    'How important is employee satisfaction': 2,     # Question 3
    'How important is product responsibility (Data privacy, Responsible Marketing, Product Quality)': 3,    # Question 4
    'How important is shareholder satisfaction': 2,  # Question 5
    'How important is a high community score (Respecting business ethics, protecting public health, and being a good citizen)': 1,           # Question 6
    'How important is best practices and corporate governance to you': 3,       # Question 7
    'What is the risk you are willing to take': 4,       # Question 8
    'How flexible are you with your preferences in stocks': 5   # Question 9
}

# Mapping user preferences 
preference_to_column_mapping = {
    'How important is environmental protection to you': 'Environment Pillar Score',
    'How important are human rights protection to you': 'Human Rights Score',
    'How important is employee satisfaction': 'Workforce Score',
    'How important is product responsibility (Data privacy, Responsible Marketing, Product Quality)': 'Product Responsibility Score',
    'How important is shareholder satisfaction': 'Shareholders Score',
    'How important is a high community score (Respecting business ethics, protecting public health, and being a good citizen)': 'Community Score',
    'How important is best practices and corporate governance to you': 'Governance Pillar Score'
}

# Filtering based on user preferences
flexibility = user_preferences['How flexible are you with your preferences in stocks']
for key, value in user_preferences.items():
    if key in ['How flexible are you with your preferences in stocks', 'What is the risk you are willing to take']:
        continue
    column = preference_to_column_mapping.get(key)
    if value == 1:
        continue 
    elif value == 2:
        threshold = averaged_data[column].quantile(0.25 - 0.05 * (flexibility - 1))
    elif value == 3:
        threshold = averaged_data[column].quantile(0.50 - 0.05 * (flexibility - 1))
    elif value == 4:
        threshold = averaged_data[column].quantile(0.70 - 0.05 * (flexibility - 1))
    elif value == 5:
        threshold = averaged_data[column].quantile(0.90 - 0.05 * (flexibility - 1))
    averaged_data = averaged_data[averaged_data[column] >= threshold]

# Filtering on risk question
risk_preference = user_preferences['What is the risk you are willing to take']
if risk_preference > 1:
    risk_threshold = data['Standard Deviation'].quantile(min(1, 0.20 * risk_preference))
    averaged_data = averaged_data[averaged_data['Standard Deviation'] <= risk_threshold]

# Top stocks based on Predicted Total Returns
top_100_stocks = averaged_data.groupby('Symbol')['Total Returns'].mean().nlargest(100).reset_index()
top_100_stocks['Predicted Total Returns'] = top_100_stocks['Total Returns'] * 10
top_100_stocks['Predicted Total Returns'] = top_100_stocks['Predicted Total Returns'].apply(lambda x: f"{x:.2f}%")
print(f"Top stocks based on questionnaire \n{top_100_stocks[['Symbol', 'Predicted Total Returns']]}")


# Load the dataset
data_path = r"C:\Users\humza\OneDrive - Worcester Polytechnic Institute (wpi.edu)\mqp\Data\Refinitiv ESG - Final Data for Analysis(Final_Table).csv"
df = pd.read_csv(data_path)

# Convert scores to numeric
score_columns = ['Emissions Score', 'Governance Pillar Score', 'Product Responsibility Score', 'Social Pillar Score', 'Human Rights Score', 'Total Returns']
df[score_columns] = df[score_columns].apply(pd.to_numeric, errors='coerce')

# Group by 'Symbol' and calculate the average for each feature
df_grouped = df.groupby('Symbol')[score_columns].mean().reset_index()

# Round the specified scores to the tenth
columns_to_round = ['Emissions Score', 'Governance Pillar Score', 'Product Responsibility Score', 'Social Pillar Score', 'Human Rights Score']
df_grouped[columns_to_round] = df_grouped[columns_to_round].round(1)

# Multiply the 'Total Returns' values by 10
df_grouped['Total Returns'] = (df_grouped['Total Returns'] * 10).round(2)
df_grouped['Total Returns'] = df_grouped['Total Returns'].astype(str) + '%'

# Calculate the 75th percentile 
emissions_75th_percentile = df_grouped['Emissions Score'].quantile(0.75)
gov_75th_percentile = df_grouped['Governance Pillar Score'].quantile(0.75)
product_75th_percentile = df_grouped['Product Responsibility Score'].quantile(0.75)
social_75th_percentile = df_grouped['Social Pillar Score'].quantile(0.75)
human_75th_percentile = df_grouped['Human Rights Score'].quantile(0.75)

# Filter symbols 
emissions_filtered_stocks = df_grouped[df_grouped['Emissions Score'] >= emissions_75th_percentile]
gov_filtered_stocks = df_grouped[df_grouped['Governance Pillar Score'] >= gov_75th_percentile]
product_filtered_stocks = df_grouped[df_grouped['Product Responsibility Score'] >= product_75th_percentile]
social_filtered_stocks = df_grouped[df_grouped['Social Pillar Score'] >= social_75th_percentile]
human_filtered_stocks = df_grouped[df_grouped['Human Rights Score'] >= human_75th_percentile]

# Top 20 stocks
top_20_stocks_emissions = df_grouped.sort_values(by='Emissions Score', ascending=False).head(20)
top_20_stocks_gov = df_grouped.sort_values(by='Governance Pillar Score', ascending=False).head(20)
top_20_stocks_product = df_grouped.sort_values(by='Product Responsibility Score', ascending=False).head(20)
top_20_stocks_social = df_grouped.sort_values(by='Social Pillar Score', ascending=False).head(20)
top_20_stocks_human = df_grouped.sort_values(by='Human Rights Score', ascending=False).head(20)

# Print the top 20 stocks with the highest scores
print("\nTop 20 stocks with the highest 'Emissions Score':")
print(top_20_stocks_emissions[['Symbol', 'Emissions Score', 'Total Returns']])

print("\nTop 20 stocks with the highest 'Governance Pillar Score':")
print(top_20_stocks_gov[['Symbol', 'Governance Pillar Score', 'Total Returns']])

print("\nTop 20 stocks with the highest 'Product Responsibility Score':")
print(top_20_stocks_product[['Symbol', 'Product Responsibility Score', 'Total Returns']])

print("\nTop 20 stocks with the highest 'Social Pillar Score':")
print(top_20_stocks_social[['Symbol', 'Social Pillar Score', 'Total Returns']])

print("\nTop 20 stocks with the highest 'Human Rights Score':")
print(top_20_stocks_human[['Symbol', 'Human Rights Score', 'Total Returns']])

Top stocks based on questionnaire 
   Symbol Predicted Total Returns
0    NXPI                   2.68%
1    SBUX                   2.58%
2      MO                   2.20%
3     HLT                   1.95%
4     AWK                   1.90%
5     HPQ                   1.70%
6     TXN                   1.66%
7     MSI                   1.65%
8       A                   1.64%
9     APD                   1.61%
10   MSFT                   1.60%
11    BDX                   1.31%
12    CVS                   1.31%
13    ABT                   1.26%
14    JPM                   1.23%
15    TGT                   1.15%
16   CSCO                   1.05%
17    MRK                   1.00%
18    PEP                   1.00%
19     WY                   0.91%
20    JNJ                   0.89%
21     KO                   0.87%
22    MMM                   0.84%
23    BAX                   0.83%
24     CL                   0.81%
25    BMY                   0.78%
26    XOM                   0.77%
27    PFE    